# 4.4: Inspecting the `decode_test_dir` and `experiment` directories

`run_test.sh` 파일은 `decode_test_dir`이라는 디렉토리를 `exp` 디렉토리 하위에 있는 `monophones`, `triphones`, 그리고 `triphones_lda` 디렉토리 아래에 생성합니다.

In [1]:
ls exp/monophones/decode_test_dir

kaldi_config_args.json  lat.2.gz  lat.4.gz  num_jobs  scoring  wer_11  wer_8
lat.1.gz                lat.3.gz  log       runtime   wer_10   wer_12  wer_9


In [2]:
ls exp/triphones/decode_test_dir

kaldi_config_args.json  lat.2.gz  lat.4.gz  num_jobs  scoring  wer_11  wer_8
lat.1.gz                lat.3.gz  log       runtime   wer_10   wer_12  wer_9


In [3]:
ls exp/triphones_lda/decode_test_dir

kaldi_config_args.json  lat.2.gz  lat.4.gz  num_jobs  scoring  wer_11  wer_8
lat.1.gz                lat.3.gz  log       runtime   wer_10   wer_12  wer_9


그 이외에 `exp/experiments/default_experiment` 디렉토리 하위에 일부 추가적인 파일을 저장하고, `decode_test_dir`의 사본도 저장합니다. 

In [4]:
ls exp/experiments/default_experiment_monophones

decode_test_dir  predicted_transcripts_10.txt  results.txt


In [5]:
ls exp/experiments/default_experiment_triphones

decode_test_dir  predicted_transcripts_10.txt  results.txt


In [6]:
ls exp/experiments/default_experiment_triphones_lda

decode_test_dir  predicted_transcripts_10.txt  results.txt


그럼 `default_experiment_monophones` 디렉토리에 있는 내용을 확인해보겠습니다. 

## `results.txt`

`results.txt` 파일에는 `word error rate (WER)`과 `sentence error rate (SER)` 정보가 들어있습니다. 

 `WER` = the percentage of words that were transcribed **incorrectly**.
 
 `SER` = the percentage of audio samples with **at least one** error in them.

In [7]:
cat exp/experiments/default_experiment_monophones/results.txt

exp/monophones/decode_test_dir/wer_10:%WER 60.97 [ 3542 / 5809, 241 ins, 739 del, 2562 sub ] [PARTIAL]
exp/monophones/decode_test_dir/wer_11:%WER 61.30 [ 3561 / 5809, 231 ins, 777 del, 2553 sub ] [PARTIAL]
exp/monophones/decode_test_dir/wer_12:%WER 61.77 [ 3588 / 5809, 226 ins, 818 del, 2544 sub ] [PARTIAL]
exp/monophones/decode_test_dir/wer_8:%WER 60.85 [ 3535 / 5809, 298 ins, 617 del, 2620 sub ] [PARTIAL]
exp/monophones/decode_test_dir/wer_9:%WER 61.06 [ 3547 / 5809, 274 ins, 679 del, 2594 sub ] [PARTIAL]

exp/monophones/decode_test_dir/wer_10:%SER 99.20 [ 248 / 250 ]
exp/monophones/decode_test_dir/wer_11:%SER 99.20 [ 248 / 250 ]
exp/monophones/decode_test_dir/wer_12:%SER 99.20 [ 248 / 250 ]
exp/monophones/decode_test_dir/wer_8:%SER 99.60 [ 249 / 250 ]
exp/monophones/decode_test_dir/wer_9:%SER 99.60 [ 249 / 250 ]


`WER`은 `minimum edit distance` 알고리즘을 사용하여서 계산된 점수입니다. 해당 알고리즘에 대한 설명은 [이 곳](https://web.stanford.edu/class/cs124/lec/med.pdf)에서 확인하실 수 있습니다.

### `minimum edit distance`

이 알고리즘은 **세 가지** 종류의 에러를 포함하고 있습니다. `kaldi`에서 점수를 매길 때에는 세 에러의 `weight`를 **동일하게** 계산합니다. 이 `weight`는 수정할 수 있습니다. 

 - substitution: replacing the correct word with a new word
     - gold: `The cat is sleeping.`
     - predicted: `The cat is yawning.`
 - insertion: inserting an **extra** word
     - gold: `The cat is sleeping.`
     - predicted: `The big cat is sleeping.`
 - deletion: removing a word
     - gold: `The cat is sleeping.`
     - predicted: `The cat sleeping.`
     

`WER` 오른쪽에는 각 타입의 에러가 몇 번 발생했는지 출력됩니다. 

```
[total_num_errors / total words, num_insertions, num_deletions, num_substitutions]
```

**Note:** 앞에서 decoding 시간 절약을 위해 전체 음성 파일의 일부만을 사용하였기 때문에 `[PARTIAL]`이라고 볼 수 있습니다. 

또한 **다섯**개의 스코어를 확인할 수 있습니다. 그 이유는 다섯개의 서로 다른 weight를 사용하여 `HCLG.fst`를 통과시켰기 때문입니다. 

### `language model weight` (`lmwt`) v. `acoustic model weight` (`acwt`)

`kaldi`에서는 `lmwt`를 `int`로 표시합니다. 만약 `lmwt=10`이라고 설정을 하게 되면, `language model`의 가중치를 `acoustic model`보다 10배 높게 설정한다는 서입니다. 만약 `lmwt=5`라고 설정을 하게되면, language model은 `acoustic model`보다 다섯 배 높은 가중치를 가지게 됩니다. 가장 낮게 설정 가능한 `lmwt`는 `lmwt=1`입니다. 이 경우, language model과 acoustic model은 동등한 가중치를 가지게 됩니다. 

`kaldi`에서 설정한 기본값은 `lmwt=10`입니다. 즉, `kaldi`에서는 기본적으로 language model을 더 중요하게 생각한다는 것을 알 수 있습니다. 

`kaldi_config.json` 파일에서 `lmwt` 값을 설정할 수 있습니다. Decoding 단계에서는 설정한 `lmwt` 값의 `+/- 2`의 weight를 사용하여서 **다섯**개의 결과값을 생성합니다. 그렇기 때문에, `lmwt` 값이 결과에 어떠한 영향을 미치는지도 대략적으로 확인할 수 있습니다. 

## `predicted_transcripts_10.txt`

이 파일에는 주어진 음성 파일을 `ASR`을 이용하여 전사한 결과가 들어있습니다. 하지만 이 경우, 결과값은 설정한 `lmwt` 값에 해당하는 하나의 결과만 들어있습니다. 

In [8]:
head exp/experiments/default_experiment_monophones/predicted_transcripts_10.txt

1089-134686-0000 HE HOPED THERE WOULD BE STUPOR DINNER TURNIPS AND CARROTS TENEBREUSE POTATOES AND THAT MONTH CEASES TO BE LABEL THAT INSISTED OPPRESSOR SECOND SAUCE 
1089-134686-0001 STARTED IN TO USE HIS BELLY COUNTED THEM 
1089-134686-0002 AFTER THE NIGHT FALLS THE LAMPS WITH LIGHT UP HERE AND THEY ARE THE SQUALID QUARTER OF THE PROPHETS 
1089-134686-0003 OH BETTY UNABLE TO TEAR MY IT 
1089-134686-0004 NOW BUT THEN FRESH KNOLLYS WAITING ON YOU CANNOT HUSBAND 
1089-134686-0005 THE MUSIC CAME NEARER AND HERE CALLED WORDS WORDS THAT SHELLEY SIGHT MET UPON THE MOMENT WANDERING COMPANIONLESS PALE FORWARDNESS 
1089-134686-0006 THE DON'T LIKE IT THOUGH MORE FAINTLY UPON THE STAGE WHERE ON ANOTHER QUESTION AGAINST ONE FOLDED SO SLOWLY AND SPREAD OF ROLLED ITS WINDING TRAIL 
1089-134686-0007 THE OLD LUCID INDIFFERENCE OR REINED IN HIS SOUL 
1089-134686-0008 THE CAME OFTEN WHICH HIS ARDOR EXTINGUISHED THE SELF WAS A COLD INDIFFERENT KNOWLEDGE OF HIMSELF 
1089-134686-0009 AT MOST A FINE WANTED

실제 전사 내용은 `raw_data/librispeech-transcripts.txt` 파일에 들어있습니다. 

In [9]:
head raw_data/librispeech-transcripts.txt

1272-128104-0000 MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL
1272-128104-0001 NOR IS MISTER QUILTER'S MANNER LESS INTERESTING THAN HIS MATTER
1272-128104-0002 HE TELLS US THAT AT THIS FESTIVE SEASON OF THE YEAR WITH CHRISTMAS AND ROAST BEEF LOOMING BEFORE US SIMILES DRAWN FROM EATING AND ITS RESULTS OCCUR MOST READILY TO THE MIND
1272-128104-0003 HE HAS GRAVE DOUBTS WHETHER SIR FREDERICK LEIGHTON'S WORK IS REALLY GREEK AFTER ALL AND CAN DISCOVER IN IT BUT LITTLE OF ROCKY ITHACA
1272-128104-0004 LINNELL'S PICTURES ARE A SORT OF UP GUARDS AND AT EM PAINTINGS AND MASON'S EXQUISITE IDYLLS ARE AS NATIONAL AS A JINGO POEM MISTER BIRKET FOSTER'S LANDSCAPES SMILE AT ONE MUCH IN THE SAME WAY THAT MISTER CARKER USED TO FLASH HIS TEETH AND MISTER JOHN COLLIER GIVES HIS SITTER A CHEERFUL SLAP ON THE BACK BEFORE HE SAYS LIKE A SHAMPOOER IN A TURKISH BATH NEXT MAN
1272-128104-0005 IT IS OBVIOUSLY UNNECESSARY FOR US TO POINT OUT HOW LUMINOUS THESE CRITICI

특정 음성 파일의 **ASR을 이용한 전사**와 **실제 전사**를 비교하기 위해서는 다음과 같은 명령어를 사용할 수 있습니다. 

In [10]:
id=1089-134686-0000    # choose the audio sample's utterance id
predicted=exp/experiments/default_experiment_monophones/predicted_transcripts_10.txt  # choose which predicted file

gold=raw_data/librispeech-transcripts.txt
diff <(cat ${gold} | grep ${id}) <(cat ${predicted} | grep ${id})

1c1
< 1089-134686-0000 HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FATTENED SAUCE
---
> 1089-134686-0000 HE HOPED THERE WOULD BE STUPOR DINNER TURNIPS AND CARROTS TENEBREUSE POTATOES AND THAT MONTH CEASES TO BE LABEL THAT INSISTED OPPRESSOR SECOND SAUCE 


: 1

먼저 출력되는 내용은 **실제 전사** 내용이며, 다음에 출력되는 내용은 **ASR을 이용한 전사** 내용입니다. 

위의 결과에서 볼 수 있듯이, 잘못 전사된 것은 왜 잘못 전사되었는지 어느 정도 이해할 수 있습니다. 예를 들어, `"stupor"`은 `"stew for"`과 비슷하게 들릴 수 있으며, `"fat mutton pieces"`는 `"that month ceases"`와 그렇게 크게 다르지는 않게 발화됩니다. 


## `decode_test_dir`

`decode_test_dir`은 `run_test.sh` 파일을 실행하여 생성된 디렉토리의 사본입니다. 어떠한 acoustic model을 사용하였는지와 무관하게 같은 종류의 파일들이 저장됩니다. 그렇기 때문에, `monophones`에 위치한 디렉토리만 한 번 살펴보겠습니다. 

In [11]:
ls exp/experiments/default_experiment_monophones/decode_test_dir

kaldi_config_args.json  lat.2.gz  lat.4.gz  num_jobs  scoring  wer_11  wer_8
lat.1.gz                lat.3.gz  log       runtime   wer_10   wer_12  wer_9


### `num_jobs`

`num_jobs`라는 파일은 `kaldi` 디렉토리에서 종종 볼 수 있습니다. 보통 하나의 `숫자`로 표시되며, `병렬처리`에서 몇 개의 thread를 사용하였는지 나타냅니다. 

In [12]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/num_jobs

4


### `runtime`

`runtime` 파일은 decoding에 소요된 **시간**을 나타냅니다. 이 파일은 `분:초`의 포맷으로 출력됩니다. 결국 최적화를 하기 위해서는 소요되는 시간과 정확성을 **동시에** 확보하여야 합니다. 

In [13]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/runtime

3:52


### `wer_*`

각각의 weight를 이용하여 산출된 결과를 저장한 결과입니다. 하나로 합쳐진 결과는 `results.txt` 파일에서 확인하 수 있습니다. 

In [14]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/wer_10

compute-wer --text --mode=present ark:exp/monophones/decode_test_dir/scoring/test_filt.txt ark,p:- 
%WER 60.97 [ 3542 / 5809, 241 ins, 739 del, 2562 sub ] [PARTIAL]
%SER 99.20 [ 248 / 250 ]
Scored 250 sentences, 2370 not present in hyp.


### `lat.*.gz`

`num_jobs`의 갯수만큼 `lat.*.gz` 파일이 존재한다는 것을 확인할 수 있습니다. 이 파일은 실제 `ASR`을 진행하는 동안 생성된 `lattice(graph)`라고 볼 수 있습니다. 

#### lattices

`lattice`는 `graph`와 같다고 볼 수 있습니다. 

이 경우, `lattice`는 단어 레벨에서 통합된 `HCLG.fst` 파일이라고 볼 수 있습니다. 이 파일은 음성 파일을 해독하기 위해서 사용 가능한 경로의 수를 나타내며, `kaldi_config.json` 파일에 있는 `max_active`와 `beam` 설정값에 의해서 크기가 결정됩니다. 

`lattice`가 생성된 이후에는 `kaldi` 알고리즘은 가장 짧은 경로를 찾아 그 결과를 예측값으로 출력합니다. 

`lattice`를 시각화 한 결과가 `resource_files/lattices`에 있습니다. 

In [15]:
ls resource_files/lattices
ls resource_files/lattices/mono
ls resource_files/lattices/tri_lda

mono  tri_lda
mono.svg  mono_zoomed.png
tri_lda.svg  tri_lda_zoom.png


아래는 `1089-134686-0000` 음성 파일을 해독하기 위해 생성된 `lattice`입니다. 

![](resource_files/lattices/mono/mono.svg "mono lattice")

전체 `lattice`는 매우 거대합니다. 그러므로 확대하여 일부 부분을 살펴보도록 하겠습니다. 

![mono_lattice_zoomed](resource_files/lattices/mono/mono_zoomed.png)

여전히 `arc`의 내용은 확인하기 어렵습니다. 하지만 각각의 `arc`는 단어와 그 단어의 `weight`를 값으로 가집니다. 

`lattice`의 크기가 큰 이유는, `lattice`를 구축하기 위해 사용한 `hyperparameter`의 영향도 있지만, 또한 주어진 모델에서 쉽게 음성 파일을 해독하는 것이 어렵다는 것을 나타냅니다. 즉, 모델에서 주어진 음성 파일을 해독할 수 있는 경우의 수가 많을 경우 `lattice`는 커집니다. 

`monophone` 모델(`WER =~ 55%)`에서 생성된 `lattice`와 `triphone_lda` 모델(`WER =~ 30%`)에서 생성된 `lattice`를 비교해보겠습니다. 더 낮은 `WER`을 보이는 모델에서 생성된 `lattice`가 더 작다는 것을 확인할 수 있습니다. 

![](resource_files/lattices/tri_lda/tri_lda.svg "tri_lda lattice")

![tri_lda_lattice_zoomed](resource_files/lattices/tri_lda/tri_lda_zoom.png)

### `decode_test_dir/scoring`

이 디렉토리에는 `ASR`의 성능을 측정하기 위한 과정에서 필요한 파일들이 저장되어 있습니다. 

In [16]:
ls exp/experiments/default_experiment_monophones/decode_test_dir/scoring

10.tra  11.tra  12.tra  8.tra  9.tra  log  test_filt.txt


#### `test_filt.txt`

이 파일에는 text 디렉토리에 있는 음성 파일들의 **실제 전사** 내용이 들어있습니다. 일반적으로 `kaldi`에서는 `[NOISE]`나 `[LAUGHTER]`와 같은 비언어적 표현에 대한 전사는 성능을 측정할 때 반영하지 않습니다. 그러므로 이 내용들은 **실제 전사**에서도 제거됩니다. 

#### `*.tra`

이 파일들은 각각의 음성 파일을 **ASR로 전사한 내용**을 담고 있습니다. 즉, 구축한 `lattice`에서 가장 짧은 경로를 통해 생성한 결과입니다. 

In [17]:
head exp/experiments/default_experiment_monophones/decode_test_dir/scoring/10.tra

1089-134686-0000 78816 82742 176142 197112 13843 169895 47587 182466 5409 27402 175011 138096 5409 175808 116456 28596 178311 13843 97712 175808 87961 126048 156992 154642 
1089-134686-0001 167586 86217 178311 187175 81503 15135 38745 175982 
1089-134686-0002 2189 175859 121875 59890 175859 98503 196097 101962 186766 80188 5409 176280 7587 175859 166822 142469 124780 175859 140508 
1089-134686-0003 125011 16336 183592 178311 174399 119158 89697 
1089-134686-0004 123281 24387 176008 66011 96176 191811 125553 198710 26149 84281 
1089-134686-0005 175859 118866 25723 120458 5409 80188 25453 196860 196860 175808 159482 161020 112911 186866 175859 115838 192226 35423 128849 65090 
1089-134686-0006 175859 49766 102058 89697 176792 116892 59625 186866 175859 167124 194350 125553 6090 142732 2243 125606 64105 163926 163061 5409 166579 124780 150369 89811 195634 179941 
1089-134686-0007 175859 125191 104742 86827 126096 146537 86217 81503 165120 
1089-134686-0008 175859 25723 124916 194417 81503

각 줄의 시작이 `utterance_id`임을 볼 수 있습니다. 하지만 실제 단어 대신에 숫자로 표시가 되어있습니다. 이 숫자들은 `graph/words.txt`에서 보이는 `index`와 동일합니다. 그러므로 `utils/int2sym.pl` 파일을 활용하여서 index를 실제 단어로 변환할 수 있습니다. 


In [18]:
head exp/experiments/default_experiment_monophones/decode_test_dir/scoring/10.tra | \
    utils/int2sym.pl -f 2- \
    exp/monophones/graph/words.txt

1089-134686-0000 HE HOPED THERE WOULD BE STUPOR DINNER TURNIPS AND CARROTS TENEBREUSE POTATOES AND THAT MONTH CEASES TO BE LABEL THAT INSISTED OPPRESSOR SECOND SAUCE 
1089-134686-0001 STARTED IN TO USE HIS BELLY COUNTED THEM 
1089-134686-0002 AFTER THE NIGHT FALLS THE LAMPS WITH LIGHT UP HERE AND THEY ARE THE SQUALID QUARTER OF THE PROPHETS 
1089-134686-0003 OH BETTY UNABLE TO TEAR MY IT 
1089-134686-0004 NOW BUT THEN FRESH KNOLLYS WAITING ON YOU CANNOT HUSBAND 
1089-134686-0005 THE MUSIC CAME NEARER AND HERE CALLED WORDS WORDS THAT SHELLEY SIGHT MET UPON THE MOMENT WANDERING COMPANIONLESS PALE FORWARDNESS 
1089-134686-0006 THE DON'T LIKE IT THOUGH MORE FAINTLY UPON THE STAGE WHERE ON ANOTHER QUESTION AGAINST ONE FOLDED SO SLOWLY AND SPREAD OF ROLLED ITS WINDING TRAIL 
1089-134686-0007 THE OLD LUCID INDIFFERENCE OR REINED IN HIS SOUL 
1089-134686-0008 THE CAME OFTEN WHICH HIS ARDOR EXTINGUISHED THE SELF WAS A COLD INDIFFERENT KNOWLEDGE OF HIMSELF 
1089-134686-0009 AT MOST A FINE WANTED

index를 실제 단어로 변환하면 `10.tra` 파일의 내용은 `predicted_transcripts_10.txt` 파일의 내용과 **완전히 일치**하게 됩니다. 

#### `decode_test_dir/scoring/log`

`scoring` 디렉토리에는 `log` 파일들이 들어있습니다. 

In [19]:
ls exp/experiments/default_experiment_monophones/decode_test_dir/scoring/log

best_path.10.log  best_path.12.log  best_path.9.log  score.11.log  score.8.log
best_path.11.log  best_path.8.log   score.10.log     score.12.log  score.9.log


`best_path.*.log` 파일에는 `C++` 명령어를 이용하여 `lattice`에서 최적의 경로를 찾아내기 위한 과정들과 그 출력이 들어있습니다. 

In [20]:
head exp/experiments/default_experiment_monophones/decode_test_dir/scoring/log/best_path.10.log

# lattice-scale --inv-acoustic-scale=10 "ark:gunzip -c exp/monophones/decode_test_dir/lat.*.gz|" ark:- | lattice-add-penalty --word-ins-penalty=0.0 ark:- ark:- | lattice-best-path --word-symbol-table=exp/monophones/graph/words.txt ark:- ark,t:exp/monophones/decode_test_dir/scoring/10.tra 
# Started at Wed Jun 27 01:24:24 UTC 2018
#
lattice-best-path --word-symbol-table=exp/monophones/graph/words.txt ark:- ark,t:exp/monophones/decode_test_dir/scoring/10.tra 
lattice-scale --inv-acoustic-scale=10 'ark:gunzip -c exp/monophones/decode_test_dir/lat.*.gz|' ark:- 
lattice-add-penalty --word-ins-penalty=0.0 ark:- ark:- 
LOG (lattice-best-path[5.2.380~1-8e7d2]:main():lattice-best-path.cc:99) For utterance 1089-134686-0000, best cost 254.32 + 9990.04 = 10244.4 over 1042 frames.
1089-134686-0000 HE HOPED THERE WOULD BE STUPOR DINNER TURNIPS AND CARROTS TENEBREUSE POTATOES AND THAT MONTH CEASES TO BE LABEL THAT INSISTED OPPRESSOR SECOND SAUCE 
LOG (lattice-best-path[5.2.380~1-8e7d2]:main():lattice

**세 가지** 명령어가 사용되었음을 확인할 수 있습니다. 

 - `lattice-scale` which sets the weights on the `lattice` (`inv-acoustic-scale` == `lmwt`)
 - `lattice-best-path` which finds the best path
 - `lattice-add-penalty` which can add a penalty for word-insertions.  Note that in this case the `word-ins-penalty` is set to `0.0`, so no adjustment is made.

`score.*.log`는 `WER` 값을 측정하기 위해 `compute-wer`로 전송되는 명령어를 보여줍니다. 

In [21]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/scoring/log/score.10.log

# cat exp/monophones/decode_test_dir/scoring/10.tra | /scratch/kaldi/egs/INSTRUCTIONAL/utils/int2sym.pl -f 2- exp/monophones/graph/words.txt | compute-wer --text --mode=present ark:exp/monophones/decode_test_dir/scoring/test_filt.txt ark,p:- >& exp/monophones/decode_test_dir/wer_10 
# Started at Wed Jun 27 01:24:32 UTC 2018
#
# Accounting: time=1 threads=1
# Ended (code 0) at Wed Jun 27 01:24:33 UTC 2018, elapsed time 1 seconds


### `decode_test_dir/log`

`decode_test_dir`은 실제 `log` 파일 또한 담고 있습니다. 

In [22]:
ls exp/experiments/default_experiment_monophones/decode_test_dir/log

analyze_alignments.log           get_lattice_stats.2.log
analyze_lattice_depth_stats.log  get_lattice_stats.3.log
decode.1.log                     get_lattice_stats.4.log
decode.2.log                     lattice_best_path.1.log
decode.3.log                     lattice_best_path.2.log
decode.4.log                     lattice_best_path.3.log
get_lattice_stats.1.log          lattice_best_path.4.log


살펴볼만한 부분은 `analyze_alignments.log` 파일과 `analyze_lattice_depth_stats.log` 파일입니다. 이 두 `log` 파일은 `run_test.sh` 스크립트를 실행할 때 사용되는 특정 부분의 실행 결과입니다. 

#### `analyze_alignments.log`

이 `log`는 각각의 `phone`의 통계값을 보여줍니다.

 - the percent of time this phone appears in the `lattice` predictions
    - `"Overall, * accounts for *% of phone occurences..."`
 - the median, mean, and 95th percentile for the number of frames this phone "required"
    - `"with duration (median, mean, 95-percentile) is (*, *, *) frames"`

In [23]:
head exp/experiments/default_experiment_monophones/decode_test_dir/log/analyze_alignments.log

# gunzip -c exp/monophones/decode_test_dir/phone_stats.*.gz | steps/diagnostic/analyze_phone_length_stats.py exp/monophones/graph 
# Started at Wed Jun 27 01:24:41 UTC 2018
#
analyze_phone_length_stats.py: could not get optional-silence phone from exp/monophones/graph/phones/optional_silence.int, guessing that it's SIL from the stats. 
At utterance begin, SIL accounts for 100.0% of phone occurrences, with duration (median, mean, 95-percentile) is (42,40.3,55) frames.
At utterance end, SIL accounts for 99.6% of phone occurrences, with duration (median, mean, 95-percentile) is (41,40.1,58) frames.
Overall, nonsilence accounts for 93.4% of phone occurrences, with duration (median, mean, 95-percentile) is (8,8.9,19) frames.
Overall, SIL accounts for 6.6% of phone occurrences, with duration (median, mean, 95-percentile) is (35,36.7,80) frames.
Overall, AH0_I accounts for 4.5% of phone occurrences, with duration (median, mean, 95-percentile) is (4,4.7,9) frames.
Overall, N_I accounts for 4.2

각각의 `phone`을 `BIES`로 나누어서 보고 있음을 알 수 있습니다. 

In [24]:
tail exp/experiments/default_experiment_monophones/decode_test_dir/log/analyze_alignments.log

Overall, M_E accounts for 0.6% of phone occurrences, with duration (median, mean, 95-percentile) is (8,9.0,15) frames.
Overall, V_I accounts for 0.6% of phone occurrences, with duration (median, mean, 95-percentile) is (6,6.2,10) frames.
Overall, R_E accounts for 0.6% of phone occurrences, with duration (median, mean, 95-percentile) is (7,8.3,16) frames.
Overall, R_B accounts for 0.6% of phone occurrences, with duration (median, mean, 95-percentile) is (10,11.4,19) frames.
The optional-silence phone SIL occupies 22.5% of frames overall 
Limiting the stats to the 90.9% of frames not covered by an utterance-[begin/end] phone, optional-silence SIL occupies 14.7% of frames.
Assuming 100 frames per second, the alignments represent 0.6 hours of data, or 0.5 hours if SIL frames are excluded.
Utterance-internal optional-silences SIL comprise 4.3% of utterance-internal phones, with duration (median, mean, 95-percentile) = (28,34.6,92)
# Accounting: time=0 threads=1
# Ended (code 0) at Wed Jun 2

`SIL`에 대한 분석 내용도 있습니다. 

In [25]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/log/analyze_alignments.log | grep SIL

analyze_phone_length_stats.py: could not get optional-silence phone from exp/monophones/graph/phones/optional_silence.int, guessing that it's SIL from the stats. 
At utterance begin, SIL accounts for 100.0% of phone occurrences, with duration (median, mean, 95-percentile) is (42,40.3,55) frames.
At utterance end, SIL accounts for 99.6% of phone occurrences, with duration (median, mean, 95-percentile) is (41,40.1,58) frames.
Overall, SIL accounts for 6.6% of phone occurrences, with duration (median, mean, 95-percentile) is (35,36.7,80) frames.
The optional-silence phone SIL occupies 22.5% of frames overall 
Limiting the stats to the 90.9% of frames not covered by an utterance-[begin/end] phone, optional-silence SIL occupies 14.7% of frames.
Assuming 100 frames per second, the alignments represent 0.6 hours of data, or 0.5 hours if SIL frames are excluded.
Utterance-internal optional-silences SIL comprise 4.3% of utterance-internal phones, with duration (median, mean, 95-percentile) = (2

일반적으로 예상할 때 각각의 음성 파일(`utterance`)은 `silence`로 시작하고 끝난다고 봅니다. 그러므로 `SIL` phone이 시작과 끝에 위치할 것이라고 예측할 수 있습니다. 이 `log` 파일은 실제로 그렇게 전사가 된 경우가 얼마나 있는지도 보여줍니다. 

`"At utterance {begin, end}..."`

#### `analyze_lattice_depth_stats.log`

이 `log` 파일은 위와 비슷하지만, `lattice`의 "깊이"에 중점을 두고 있습니다. 

In [26]:
head exp/experiments/default_experiment_monophones/decode_test_dir/log/analyze_lattice_depth_stats.log

# gunzip -c exp/monophones/decode_test_dir/depth_stats_tmp.*.gz | steps/diagnostic/analyze_lattice_depth_stats.py exp/monophones/graph 
# Started at Wed Jun 27 01:24:41 UTC 2018
#
The total amount of data analyzed assuming 100 frames per second is 0.6 hours
Overall, lattice depth (10,50,90-percentile)=(3,15,59) and mean=24.5
Nonsilence phones as a group account for 77.5% of frames, with lattice depth (10,50,90-percentile)=(3,16,60) and mean=25.5
Phone SIL accounts for 22.5% of frames, with lattice depth (10,50,90-percentile)=(2,12,51) and mean=21.0
Phone N_I accounts for 2.8% of frames, with lattice depth (10,50,90-percentile)=(4,16,55) and mean=25.2
Phone S_B accounts for 2.4% of frames, with lattice depth (10,50,90-percentile)=(3,20,77) and mean=30.1
Phone T_I accounts for 2.4% of frames, with lattice depth (10,50,90-percentile)=(4,20,75) and mean=31.2


정확하게 이 `log` 파일이 나타내는 바를 해석하는 것은 쉽지 않습니다. 하지만 `kaldi` 공식 문서에서는 `lattice depth`를 "프레임을 넘어가는 `arc`의 평균 갯수"라고 정의하고 있습니다. 이 `log` 파일에서 확실하게 확인할 수 있는 것은, `monophone` 모델을 이용해서 생성된 `lattice`가 `triphone_lda`를 사용한 모델보다 훨씬 크다는 것입니다. `lattice depth`를 아래와 같이 비교해보면 실제로 `monophone`과 `triphone_lda`를 사용하여 생성된 `lattice`가 어떻게 다른지 볼 수 있습니다. 

In [27]:
head exp/experiments/default_experiment_triphones_lda/decode_test_dir/log/analyze_lattice_depth_stats.log

# gunzip -c exp/triphones_lda/decode_test_dir/depth_stats_tmp.*.gz | steps/diagnostic/analyze_lattice_depth_stats.py exp/triphones_lda/graph 
# Started at Wed Jun 27 01:39:11 UTC 2018
#
The total amount of data analyzed assuming 100 frames per second is 0.6 hours
Overall, lattice depth (10,50,90-percentile)=(1,3,8) and mean=3.9
Nonsilence phones as a group account for 79.6% of frames, with lattice depth (10,50,90-percentile)=(1,3,8) and mean=3.9
Phone SIL accounts for 20.4% of frames, with lattice depth (10,50,90-percentile)=(1,2,9) and mean=4.1
Phone S_B accounts for 2.4% of frames, with lattice depth (10,50,90-percentile)=(1,3,10) and mean=4.4
Phone N_I accounts for 2.3% of frames, with lattice depth (10,50,90-percentile)=(1,3,8) and mean=3.8
Phone T_E accounts for 2.2% of frames, with lattice depth (10,50,90-percentile)=(1,3,9) and mean=4.4
